In [1]:
#!python -m spacy download en_core_web_md

In [2]:
import spacy
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.models import load_model
#from keras.utils import to_categorical

In [3]:
BATCH_SIZE=128
SEQ_LEN=100

In [4]:
embedding_file=spacy.load('en_core_web_md')

In [5]:
doc=embedding_file("My Name Is")

In [6]:
word='like'
embedding=embedding_file(word).vector

In [7]:
len(embedding)

300

In [8]:
"""from google.colab import files
uploads=files.upload()"""

'from google.colab import files\nuploads=files.upload()'

In [11]:
train=pd.read_excel('/content/drive/MyDrive/IMDB Dataset/With_Test_Set/Small_Train.xlsx')
valid=pd.read_excel('/content/drive/MyDrive/IMDB Dataset/With_Test_Set/Small_Valid.xlsx')
test=pd.read_excel('/content/drive/MyDrive/IMDB Dataset/With_Test_Set/Small_Test.xlsx')

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
train.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [13]:
labels=['Negative','Positive']

In [14]:
x_tr,y_tr=train['text'].values,pd.get_dummies(train['label'].values).values
x_val,y_val=valid['text'].values,pd.get_dummies(valid['label'].values).values
x_test,y_test=test['text'].values,pd.get_dummies(test['label'].values).values

In [ ]:
x_tr

In [16]:
tokenizer=Tokenizer()

In [17]:
tokenizer.fit_on_texts(list(x_tr))

In [18]:
x_tr_seq=tokenizer.texts_to_sequences(x_tr)

In [19]:
x_val_seq=tokenizer.texts_to_sequences(x_val)
x_test_seq=tokenizer.texts_to_sequences(x_test)

In [20]:
x_tr_seq=pad_sequences(x_tr_seq,maxlen=SEQ_LEN)
x_val_seq=pad_sequences(x_val_seq,maxlen=SEQ_LEN)
x_test_seq=pad_sequences(x_test_seq,maxlen=SEQ_LEN)

In [21]:
size_of_vocab=len(tokenizer.word_index)+1
size_of_vocab

12816

Training Word Embeddings from Scratch

In [22]:
model=Sequential()
model.add(Embedding(size_of_vocab,300,input_length=SEQ_LEN,trainable=True))
model.add(LSTM(128,return_sequences=True,dropout=0.2))
model.add(GlobalMaxPool1D())
model.add(Dense(64,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [23]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
es=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=3)
mc=ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',save_best_only=True,verbose=1)

In [24]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          3844800   
_________________________________________________________________
lstm (LSTM)                  (None, 100, 128)          219648    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 4,072,834
Trainable params: 4,072,834
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
history=model.fit(x_tr_seq,y_tr,batch_size=BATCH_SIZE,epochs=10,validation_data=(x_val_seq,y_val),verbose=1,callbacks=[es,mc],validation_steps=1)

Epoch 1/10
4/4 [==============================] - 23s 858ms/step - loss: 0.6921 - acc: 0.5366 - val_loss: 0.6891 - val_acc: 0.5500

Epoch 00001: val_acc improved from -inf to 0.55000, saving model to best_model.h5
Epoch 2/10
4/4 [==============================] - 2s 575ms/step - loss: 0.6837 - acc: 0.5317 - val_loss: 0.6898 - val_acc: 0.5300

Epoch 00002: val_acc did not improve from 0.55000
Epoch 3/10
4/4 [==============================] - 2s 555ms/step - loss: 0.6681 - acc: 0.5917 - val_loss: 0.6923 - val_acc: 0.5300

Epoch 00003: val_acc did not improve from 0.55000
Epoch 4/10
4/4 [==============================] - 2s 562ms/step - loss: 0.6427 - acc: 0.6772 - val_loss: 0.6966 - val_acc: 0.6700

Epoch 00004: val_acc improved from 0.55000 to 0.67000, saving model to best_model.h5
Epoch 00004: early stopping


In [26]:
model=load_model('best_model.h5')

In [27]:
_,val_acc=model.evaluate(x_val_seq,y_val,batch_size=BATCH_SIZE)
print(val_acc)

1/1 [==============================] - 1s 957ms/step - loss: 0.6966 - acc: 0.6700
0.6700000166893005


In [28]:
sample_comment=['I love that I hate this movie']

In [29]:
sample_token=tokenizer.texts_to_sequences(sample_comment)

In [30]:
sample_token_pad=pad_sequences(sample_token,maxlen=100)

In [31]:
prediction=model.predict(sample_token_pad)

In [32]:
prediction

array([[0.22362632, 0.7763737 ]], dtype=float32)

In [33]:
labels[np.argmax(prediction)]

'Positive'

Using Pre-Trained GloVe Embeddings

In [34]:
"""from google.colab import files
uploads=files.upload()"""

'from google.colab import files\nuploads=files.upload()'

In [35]:
embedding_matrix=np.zeros((size_of_vocab,300))
for word,i in tokenizer.word_index.items():
  embedding_vector=embedding_file(word).vector
  if embedding_vector is not None:
    embedding_matrix[i]=embedding_vector

In [36]:
embedding_matrix[1]

array([ 2.72040009e-01, -6.20299987e-02, -1.88400000e-01,  2.32250001e-02,
       -1.81580000e-02,  6.71919994e-03, -1.38769999e-01,  1.77080005e-01,
        1.77090004e-01,  2.58820009e+00, -3.51790011e-01, -1.73120007e-01,
        4.32850003e-01, -1.07079998e-01,  1.50059998e-01, -1.99819997e-01,
       -1.90929994e-01,  1.18710005e+00, -1.62070006e-01, -2.35379994e-01,
        3.66399996e-03, -1.91560000e-01, -8.56619999e-02,  3.91989984e-02,
       -6.64490014e-02, -4.20899987e-02, -1.91220000e-01,  1.16790002e-02,
       -3.71380001e-01,  2.18860000e-01,  1.14229997e-03,  4.31899995e-01,
       -1.42049998e-01,  3.80589992e-01,  3.06540012e-01,  2.01670006e-02,
       -1.83160007e-01, -6.51860004e-03, -8.05489998e-03, -1.20630004e-01,
        2.75069997e-02,  2.98390001e-01, -2.28960007e-01, -2.28819996e-01,
        1.46709993e-01, -7.63010010e-02, -1.26800001e-01, -6.66509988e-03,
       -5.27950004e-02,  1.42580003e-01,  1.56100005e-01,  5.55099994e-02,
       -1.61489993e-01,  

In [37]:
model=Sequential()
model.add(Embedding(size_of_vocab,300,weights=[embedding_matrix],input_length=100,trainable=False))
model.add(LSTM(128,return_sequences=True,dropout=0.2))
model.add(GlobalMaxPool1D())
model.add(Dense(64,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [38]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
es=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=3)
mc=ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',save_best_only=True,verbose=1)

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          3844800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 128)          219648    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 4,072,834
Trainable params: 228,034
Non-trainable params: 3,844,800
_________________________________________________________________


In [40]:
history=model.fit(np.array(x_tr_seq),np.array(y_tr),batch_size=128,epochs=10,validation_data=(np.array(x_val_seq),np.array(y_val)),verbose=1,callbacks=[es,mc],validation_steps=1)

Epoch 1/10
4/4 [==============================] - 5s 664ms/step - loss: 0.6992 - acc: 0.5075 - val_loss: 0.6735 - val_acc: 0.6300

Epoch 00001: val_acc improved from -inf to 0.63000, saving model to best_model.h5
Epoch 2/10
4/4 [==============================] - 2s 445ms/step - loss: 0.6697 - acc: 0.5571 - val_loss: 0.6613 - val_acc: 0.6500

Epoch 00002: val_acc improved from 0.63000 to 0.65000, saving model to best_model.h5
Epoch 3/10
4/4 [==============================] - 2s 433ms/step - loss: 0.6464 - acc: 0.6767 - val_loss: 0.6582 - val_acc: 0.6400

Epoch 00003: val_acc did not improve from 0.65000
Epoch 4/10
4/4 [==============================] - 2s 433ms/step - loss: 0.6248 - acc: 0.7358 - val_loss: 0.6168 - val_acc: 0.7100

Epoch 00004: val_acc improved from 0.65000 to 0.71000, saving model to best_model.h5
Epoch 5/10
4/4 [==============================] - 2s 443ms/step - loss: 0.5746 - acc: 0.7621 - val_loss: 0.5917 - val_acc: 0.7200

Epoch 00005: val_acc improved from 0.71000 

In [41]:
model=load_model('best_model.h5')

In [42]:
_,val_acc=model.evaluate(x_test_seq,y_test,batch_size=128)
print(val_acc)

1/1 [==============================] - 1s 774ms/step - loss: 0.4568 - acc: 0.7600
0.7599999904632568


In [43]:
#sample_comment=['I do not realy hate this movie']
sample_comment=['I love that I hate this movie']
sample_token=tokenizer.texts_to_sequences(sample_comment)
sample_token_pad=pad_sequences(sample_token,maxlen=100)
prediction=model.predict(sample_token_pad)

In [44]:
labels[np.argmax(prediction)]

'Positive'

In [45]:
prediction

array([[0.40154618, 0.5984538 ]], dtype=float32)